**Однонаправленная LSTM с пословной токенизацией:**

In [ ]:
import numpy as np
import os

import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
import keras.layers as l

from keras import models, callbacks, utils, losses

In [ ]:
from typing import List
from bs4 import BeautifulSoup
import requests
import os
def request_url(url: str) -> BeautifulSoup:
    request = requests.get(url)
    soup = BeautifulSoup(request.content, 'html.parser')
    return soup


def get_url_data(url: str) -> List[str]:
    soup = request_url(url)
    scrapped_text = []

    h1 = soup.h1.text.strip()
    p = soup.find_all('p')

    scrapped_text.append(h1)
    scrapped_text.extend([p_i.text.strip() for p_i in p])

    return scrapped_text

def get_data(url: str) -> str:
    soup = request_url(url)
    text = []

    text.extend([
        soup.h1.text.strip() + '.',
        soup.h2.text.strip() + '.',
        soup.article.p.text.strip()
    ])

    url_chapters = [link.get('href') for link in soup.find_all('a', class_='link')]

    for url in url_chapters:
        scrapped_text = get_url_data(url)
        text.extend(scrapped_text)

    text = ' '.join(text).lower()

    return text

In [ ]:
def load_data(url: str, file_name: str, path_dir: str = 'data/') -> str:
    if os.path.isdir(path_dir) == False:
        os.mkdir(path_dir)
        print(f'Created {path_dir} directory')

    path_file = f'{path_dir}{file_name}'

    try:
        with open(path_file, 'r', encoding='utf-8') as file:
            text = file.read()

        print('Uploaded from', path_file)

    except:
        text = get_data(url)

        with open(path_file, 'w', encoding='utf-8') as file:
            file.write(text)

        print('Saved to', path_file)

    return text

In [ ]:
text = load_data('https://hpmor.ru/', 'hpmor.txt')

Created data/ directory
Saved to data/hpmor.txt


In [ ]:
import re
def split_into_sentences(text: str, regex: str = '[^а-яА-ЯёЁ0-9 ,-]') -> List[str]:
    sentences = [re.sub(regex, '', s).strip() for s in text.split('.')]
    sentences = list(filter(None, sentences))
    return sentences

In [ ]:
text_word = split_into_sentences(text, "[^а-яА-ЯёЁ0-9 ,-]")

In [ ]:
text_word = text_word[:1000000]

**Токенизация**

In [ ]:
dictionary = np.array(sorted(set(' '.join(text_word).split(' '))))
dictionary

array(['', ',', '-', ..., 'ёовтен', 'ёрзал', 'ёрзали'], dtype='<U70')

In [ ]:
tokenizer_word_to_index_word = {char: i for i, char in enumerate(dictionary)}
tokenizer_index_to_word_word = {i: char for i, char in enumerate(dictionary)}

**Формирование датасета:**

In [ ]:
vectorized_text = np.array([tokenizer_word_to_index_word[word] for word in ' '.join(text_word).split()])

In [ ]:
vectorized_text = Dataset.from_tensor_slices(vectorized_text)

In [ ]:
VOCAB_LEN = len(tokenizer_word_to_index_word.items())
BATCH_SIZE = 100

In [ ]:
sequences = vectorized_text.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
BATCHES_PER_EPOCH = len(sequences) // BATCH_SIZE

In [ ]:
from typing import Tuple, List
def get_features_target(seq: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    features = seq[:-1]
    target = seq[1:]
    return features, target

In [ ]:
df = sequences.map(get_features_target).batch(BATCH_SIZE, drop_remainder=True).repeat()
df = df.prefetch(AUTOTUNE)

**Построение и обучение модели:**

In [ ]:
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=len(dictionary),
    input_shape=(1,))

In [ ]:
import keras.layers as l
model = keras.Sequential([
    l.Embedding(len(dictionary), BATCH_SIZE, batch_input_shape=[BATCH_SIZE, None]),
    l.LSTM(64, return_sequences=True, stateful=True),
    l.Dense(len(dictionary))
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (100, None, 100)          6108100   
                                                                 
 lstm (LSTM)                 (100, None, 64)           42240     
                                                                 
 dense (Dense)               (100, None, 61081)        3970265   
                                                                 
Total params: 10120605 (38.61 MB)
Trainable params: 10120605 (38.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
preds = model.fit(
    df,
    epochs=100,
    steps_per_epoch=BATCHES_PER_EPOCH
)

Epoch 1/100
53/53 [==============================] - 25s 415ms/step - loss: 10.1509 - accuracy: 0.0241
Epoch 2/100
53/53 [==============================] - 17s 318ms/step - loss: 8.3611 - accuracy: 0.0264
Epoch 3/100
53/53 [==============================] - 17s 324ms/step - loss: 8.2584 - accuracy: 0.0282
Epoch 4/100
53/53 [==============================] - 18s 333ms/step - loss: 8.2516 - accuracy: 0.0282
Epoch 5/100
53/53 [==============================] - 18s 343ms/step - loss: 8.2340 - accuracy: 0.0282
Epoch 6/100
53/53 [==============================] - 19s 355ms/step - loss: 8.1836 - accuracy: 0.0282
Epoch 7/100
53/53 [==============================] - 19s 356ms/step - loss: 8.1417 - accuracy: 0.0282
Epoch 8/100
53/53 [==============================] - 18s 347ms/step - loss: 8.1047 - accuracy: 0.0283
Epoch 9/100
53/53 [==============================] - 18s 345ms/step - loss: 8.0713 - accuracy: 0.0306
Epoch 10/100
53/53 [==============================] - 19s 350ms/step - loss: 8.04